In [1]:
%load_ext autoreload
%autoreload 2

import json
import pprint

from pgoapi import PGoApi
from pgoapi.utilities import f2i
from stepper import Stepper, _get_cellid
from geopy.geocoders import GoogleV3

config = json.load(open('config.json'))

In [2]:
import random 
METER_TO_DEG = 1. / (60 * 1852)

def fuzz(pos, tolerance=10):
    lat, lon, alt = pos
    n0 = random.normalvariate(0, tolerance * METER_TO_DEG) 
    n1 = random.normalvariate(0, tolerance * METER_TO_DEG)
    return lat + n0, lon + n1, alt

In [3]:
geolocator = GoogleV3(api_key=config['gmapkey'])
loc = geolocator.geocode(config['location'])
pos = loc.latitude, loc.longitude, loc.altitude

In [4]:
api = PGoApi()
api.set_position(*fuzz(pos))
api.login(config['auth_service'], config['username'], config['password'])
api.get_player()
api.call()

{'api_url': u'pgorelease.nianticlabs.com/plfe/10',
 'auth_ticket': {'end': 'AWGtvohh4ZRL5rhVN9MHUg==\n',
  'expire_timestamp_ms': 1469233053884L,
  'start': 'a5noqN9pBoVSAgn6njvQOgY0H79moPv7au3kfUuP57rrWZpsPIC+gsuL8kKwBsN1tvT5nx+aw+xw\nf7IHgLp8fQ==\n'},
 'direction': 2,
 'responses': {'GET_PLAYER': {'profile': {'avatar': {},
    'creation_time': 1469076398123L,
    'currency': [{'type': u'POKECOIN'},
     {'amount': 15300, 'type': u'STARDUST'}],
    'daily_bonus': {},
    'item_storage': 350,
    'poke_storage': 250,
    'tutorial': 'AAEDBAc=\n',
    'unknown12': '',
    'unknown13': '',
    'username': u'Seer1025'},
   'unknown1': 1}},
 'unknown2': 8145806132888207460L}

In [ ]:
timestamp = "\000" * 21
cellid = _get_cellid(*pos[:2])
api.get_map_objects(latitude=f2i(pos[0]),
                    longitude=f2i(pos[1]),
                    since_timestamp_ms=timestamp,
                    cell_id=cellid)
api.call()

In [ ]:
api.get_inventory()
res = api.call()

In [18]:
pokemons = [item['inventory_item_data']['pokemon']
            for item in res['responses']['GET_INVENTORY']['inventory_delta']['inventory_items']
            if 'pokemon'    in item['inventory_item_data'] and
               'is_egg' not in item['inventory_item_data']['pokemon']]


In [43]:
k = 1
to_transfer = []
for pid, group in groupby(pokemons, lambda x: x['pokemon_id']):
    g = list(group)
    if len(g) > k:
        for obj in sorted(g, key=lambda x: x['cp'], reverse=True)[:k]:
            to_transfer.append(obj['id'])
to_transfer

[13482145766303099175L,
 16719760576883640229L,
 5720383337322283645L,
 14643652224305950755L,
 10080236136470070615L,
 12486898399445972262L]

In [34]:
from itertools import groupby
for pid, group in groupby(pokemons, lambda x: x['pokemon_id']):
    g = list(group)
    
        print len(g)

2
2
2
2
2
2
